In [10]:
import pandas as pd
import numpy as np

from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models import Word2Vec

In [11]:
test_df = pd.read_csv('./dataset/restaurants_data.csv', index_col=0)
menu_cuisine_df = pd.read_csv('./dataset/menu_tfidf.csv', index_col=0)

In [103]:
# Utilize helper functions
%run helper_function.ipynb

In [13]:
test_df.head(3)

,name,clean_text,menu,puerto,healthy,sandwiches,kids,barbecue,malaysian,burgers,...,hawaiian,juices,bakery,teahouses,middle,brazilian,classic,fruit,breakfast,eastern
0,#1 pho,fantastic pho vegetable pho vegetable broth fr...,finely ground pork grill wooden skewer lettuce...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,24th street pizza & gyros,change ownership stop day advertise pizza slic...,tomato onion tzatziki lettuce tomato onion tza...,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3 tomatoes & a mozzarella,a cute unassume little bistro pizza go friday ...,small salad dress house balsamic vinaigrette r...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## 1) Information Retrieval using word2vec
Generating 'word to vector' to capture contextand preserve the semantic and syntactic relationship between words.
- **Vector Space Models (VSM)** - the concept of similarity between documents using cosine similarity.

### 1.1 Information Retrieval (IR)
Returning most relevant restaurants, followed by less relevant documents (document ranking)

### 1.1 Ranked Retrieval using Word2vec based Vector Space Model (VSM)
 VSM assumes that relevance between a document and the query is directly related to the similarity between their vector representations. In other words, when user search input's vectors has a high similarity with one of the documents, it will rank the document high compared to documents with lower similarity scores.
 
 - Train Word2Vec model
 - Test Word2Vec model with search input

#### Train Word2Vec Model
- Gather text data from both text and menu columns

In [22]:
# Combine text and menu data
clean_text_data = test_df['clean_text'].tolist()
clean_menu_data = test_df['menu'].tolist()

# Training word2vc models (review text and menu description)
text_w2v_model = Word2Vec(clean_text_data, # Training data (text)
                     size=300, # Create 300 vectors
                     min_count=2, # Ignore all words with total frequency lower than 2
                     window=5, # Maximum distance between the current and predicted word within the sentence
                     sg=1, # Apply skip-gram training algorithm
                     workers=4) # Maximize worker thread for faster processing

In [23]:
# Training word2vc models (review text and menu description)
menu_w2v_model = Word2Vec(clean_menu_data, # Training data (text)
                     size=300, # Create 300 vectors
                     min_count=2, # Ignore all words with total frequency lower than 2
                     window=5, # Maximum distance between the current and predicted word within the sentence
                     sg=1, # Apply skip-gram training algorithm
                     workers=4) # Maximize worker thread for faster processing

In [101]:
# Training with combined menu and text values
test_df['combined_text'] = test_df['clean_text'] + test_df['menu']

data = test_df['combined_text'].tolist()

# Training word2vc models (review text and menu description)
w2v_model = Word2Vec(data, # Training data (text)
                     size=1000, # Create 300 vectors
                     min_count=2, # Ignore all words with total frequency lower than 2
                     window=5, # Maximum distance between the current and predicted word within the sentence
                     sg=1, # Apply skip-gram training algorithm
                     workers=4) # Maximize worker thread for faster processing

In [64]:
w2v_model.save("./dataset/word2vec.model")

In [24]:
# Saving trained model
text_w2v_model.save("./dataset/text_word2vec.model")
menu_w2v_model.save("./dataset/menu_word2vec.model")

In [65]:
# Vocabulary size
print('Text w2v vocabulary size:', len(text_w2v_model.wv.vocab))
print('Menu w2v vocabulary size:', len(menu_w2v_model.wv.vocab))
print('Main w2v vocabulary size:', len(w2v_model.wv.vocab))

Text w2v vocabulary size: 603
Menu w2v vocabulary size: 47
Main w2v vocabulary size: 603


In [102]:
print('Main w2v vocabulary size:', len(w2v_model.wv.vocab))

Main w2v vocabulary size: 603


#### Create text and menu dataframes

In [104]:
text_df = test_df[['name','clean_text']].copy()
menu_df = test_df[['name', 'menu']].copy()
main_df = test_df[['name', 'combined_text']].copy()

#### Generate average vector per document
Since the word2vec provides vectors for a word - we need to generate vectors for all the whole document. get_embedding_w2v function will average the vector for the document.

In [68]:
text_df['vector'] = test_df['clean_text'].apply(lambda x: get_embedding_w2v(x.split(), text_w2v_model))

In [69]:
text_df.head()

,name,clean_text,vector
0,#1 pho,fantastic pho vegetable pho vegetable broth fr...,"[0.521884108139489, 0.4930287262995464, 0.5050..."
1,24th street pizza & gyros,change ownership stop day advertise pizza slic...,"[0.49656892312819795, 0.49886690911446224, 0.5..."
2,3 tomatoes & a mozzarella,a cute unassume little bistro pizza go friday ...,"[0.5015492555658613, 0.5070190469564072, 0.495..."
3,4b cafe,good consensus average home fairly easily emul...,"[0.4866628651462132, 0.49869099237957937, 0.51..."
4,5 r cha thai go,try r cha today turn permanently close very sa...,"[0.4985590962896284, 0.5005264575577869, 0.489..."


In [70]:
menu_df['vector'] = test_df['menu'].apply(lambda x: get_embedding_w2v(x.split(), menu_w2v_model))

In [71]:
menu_df.head()

,name,menu,vector
0,#1 pho,finely ground pork grill wooden skewer lettuce...,"[0.504928941545973, 0.5062170667526904, 0.5147..."
1,24th street pizza & gyros,tomato onion tzatziki lettuce tomato onion tza...,"[0.4790185526942054, 0.50761180448245, 0.49167..."
2,3 tomatoes & a mozzarella,small salad dress house balsamic vinaigrette r...,"[0.5079650969721645, 0.5116258773484551, 0.496..."
3,4b cafe,stuff cream cheese freshly fry corn tortilla c...,"[0.4938781281506183, 0.5050950947907523, 0.496..."
4,5 r cha thai go,finely sliced veggie glass noodle wrap lightly...,"[0.509412049066758, 0.5023365933045508, 0.5108..."


In [105]:
main_df['vector'] = test_df['combined_text'].apply(lambda x: get_embedding_w2v(x.split(), w2v_model))

In [106]:
main_df.head()

,name,combined_text,vector
0,#1 pho,fantastic pho vegetable pho vegetable broth fr...,"[0.494963195408833, 0.5175969678686941, 0.4831..."
1,24th street pizza & gyros,change ownership stop day advertise pizza slic...,"[0.5011877695189417, 0.5064897990021704, 0.504..."
2,3 tomatoes & a mozzarella,a cute unassume little bistro pizza go friday ...,"[0.5021902525624422, 0.4991204675477027, 0.501..."
3,4b cafe,good consensus average home fairly easily emul...,"[0.5063407927553903, 0.5019632034855142, 0.488..."
4,5 r cha thai go,try r cha today turn permanently close very sa...,"[0.48486736856131274, 0.4971123247304999, 0.49..."


#### Test w2v models using sample query

In [86]:
text_result_df = w2v_ranking_ir('french', text_w2v_model, text_df, 'text')

In [87]:
text_result_df

,name,clean_text,similarity
0,italia express,this decent think sbarros locate court las veg...,0.860339
1,roma pizza,a friend recommend figure menu order pizza bre...,0.859523
2,cc's cafe,as type meal the menu eclectic spectacular dri...,0.859462
3,taste buds,gas gallon hill recently remodel believe gas p...,0.859322
4,oriental wok,the portion huge price the disappoint chicken ...,0.859244
...,...,...,...
1194,pronto pizza,let start really slow dine take hour leave the...,0.855263
1195,tony's pizzeria,do judge book cover pull tony come kitchen gol...,0.855010
1196,lucky cat,love lucky cat stop walk fortune coupon day ta...,0.854542
1197,ming's garden,the ming garden yelp exact current believe reo...,0.854525


In [82]:
menu_result_df = w2v_ranking_ir('french', menu_w2v_model, menu_df, 'menu')

In [83]:
menu_result_df

,name,menu,similarity
0,the taco spot,vegetarian friendly vegetarian friendly vegeta...,0.854769
1,la bella vita,liter flour semolina dust flash fried serve ma...,0.854680
2,shibuya,organic white wakame king nigori nameko mushro...,0.854517
3,arizona authentic pizza,with lettuce cucumber cherry tomato with lettu...,0.854321
4,ferraro's italian restaurant & wine bar,grill homemade spicy sausage roasted shallot m...,0.854303
...,...,...,...
1194,shamrock bar,two egg choice sausage bacon ham two buttermil...,0.847945
1195,kj kitchen,jelly fish squid with xo sauce lamb clay pot b...,0.847910
1196,fuji,choice items choice items salmon tofu dinner s...,0.844559
1197,honey pig,coca cola diet coke sprite stir fry kim chi bo...,0.829493


In [123]:
main_result_df = w2v_ranking_ir('sushi japanese', w2v_model, main_df, 'combined')

In [124]:
main_result_df[:10]

,name,combined_text,similarity
0,green leaf,eat indian long bcz move frm nyc s ton fantast...,0.930515
1,luciano's,darn fatty slice pizza pricey work mall mall d...,0.930484
2,la perla,this la perla store usually light airy italy r...,0.930452
3,taco express,husband hunt new place stumble tiny taco minut...,0.930430
4,divine cafe,the visit catfish poi boy delicious pack flavo...,0.930423
5,spasso pizza & mozzarella bar,favorite wood fire pizza central phoenix spass...,0.930410
6,tropicana pizza,tropicana pizza rocks the gentleman work custo...,0.930407
7,5 r cha thai go,try r cha today turn permanently close very sa...,0.930400
8,cafe on the strip,friend duck coffee sweet walk strip thing ask ...,0.930386
9,china restaurant,chinese takeout lot lately portion go downhill...,0.930362


#### Gather all text and menu information

In [8]:
text_sentences = []
menu_sentences = []

# Enusre they're all string datatype
test_df['clean_text'] = test_df['clean_text'].astype(str)
test_df['menu'] = test_df['menu'].astype(str)

# Gather all paragraphs from clean text and menu columns
test_df['clean_text'].apply(lambda text: text_sentences.append(text));
test_df['menu'].apply(lambda text: menu_sentences.append(text));

In [17]:
'''
vector_size - defining dimensions
min_count - minimum word count to 100 in order to discard words with very few occurrences
alpha - initial learning rate
min_alpha - learning rate will linearly drop to min_alpha as training progresses
dm - 1: distributed memory (PV-DM) 0: distributed bag of words (PV-DBOW)
epochs - ierating over the trainig corpus 5 times

NOTE: Distributed memory model preserves the word order in a document whereas distributed bag of words just uses
the bag of words approach, which doesn't preserve any word order
'''
menu_doc2vec_model = Doc2Vec(vector_size=100, min_count=10, dm=1, epochs=20)
text_doc2vec_model = Doc2Vec(vector_size=100, min_count=10, dm=1, epochs=20)

In [18]:
text_tagged_data = [TaggedDocument(words=sentence, tags=[str(i)]) for i, sentence in enumerate(text_sentences)]
menu_tagged_data = [TaggedDocument(words=sentence, tags=[str(i)]) for i, sentence in enumerate(menu_sentences)]

In [19]:
menu_doc2vec_model.build_vocab(menu_tagged_data)
text_doc2vec_model.build_vocab(text_tagged_data)

In [20]:
menu_doc2vec_model.train(menu_tagged_data, total_examples=menu_doc2vec_model.corpus_count, epochs=menu_doc2vec_model.epochs)
text_doc2vec_model.train(text_tagged_data, total_examples=text_doc2vec_model.corpus_count, epochs=text_doc2vec_model.epochs)

In [21]:
np.arange(0,100,10)

array([ 0, 10, 20, 30, 40, 50, 60, 70, 80, 90])

In [76]:
def train_doc2vec(doc2vec_model,tagged_data):
    for epoch in np.arange(0,100,10): # Train the model 5 times with decreasing learning rate

        doc2vec_model.train(tagged_data,
                    total_examples=doc2vec_model.corpus_count,
                    epochs=doc2vec_model.iter)

        # decrease the learning rate
        doc2vec_model.alpha -= 0.001
        # fix the learning rate, no decay
        doc2vec_model.min_alpha = doc2vec_model.alpha

train_doc2vec(menu_doc2vec_model, menu_tagged_data)

train_doc2vec(text_doc2vec_model, text_menu_data)

In [23]:
menu_doc2vec_model.save('./dataset/menu_d2v.model')

In [24]:
text_doc2vec_model.save('./dataset/text_d2v.model')

### Sample Inpput

In [25]:
# Test input
test_value = 'Restaurant with chinese food'

# Populate vectors based on 'menu' doc2vec model
menu_vector = menu_doc2vec_model.infer_vector(test_value.split())

# Populate vectors based on 'text' doc2vec model
text_vector = text_doc2vec_model.infer_vector(test_value.split())

# Sample output
menu_vector

array([ 1.3068430e-03, -6.7983364e-04,  3.0254084e-03,  1.3698388e-03,
       -2.0751443e-03, -2.9299206e-03, -3.1833653e-03,  4.9149697e-03,
        3.0099874e-04, -2.5248916e-03,  4.8288726e-03,  4.7107609e-03,
       -4.6823560e-03,  9.8085776e-04, -4.7048470e-03, -4.3320036e-03,
       -2.2970943e-03, -1.6628477e-03,  3.1944856e-03,  3.1066660e-03,
        8.0796471e-04, -4.8057823e-03,  2.4536813e-03,  3.3315173e-03,
       -3.1034409e-03, -1.7111792e-03, -1.5654387e-03, -1.5256809e-03,
       -4.6807178e-03,  9.8433066e-04,  4.7049108e-03, -9.0547453e-04,
       -1.5206836e-03, -3.0542060e-03,  7.7218318e-04, -2.2456083e-03,
       -2.4128731e-03,  2.5712454e-03, -1.3377758e-03, -4.2135091e-04,
        3.3068209e-04, -4.2986306e-03,  2.9677795e-03, -4.0228311e-03,
        9.7742386e-04,  3.5818003e-03,  6.3398929e-04,  1.3683768e-03,
       -1.4692416e-03, -3.9688936e-03, -4.3332023e-03,  4.6121939e-03,
        2.7100611e-04,  3.1390919e-03,  3.2444277e-03,  5.4585899e-04,
      

In [65]:
test_df.head(1)

,name,clean_text,menu,vegan,asian,ribs,fine,tapas,bistro,szechwan,...,fresh,puerto,gyros,cantonese,european,cream,cocktails,pub,indian,steak
0,#1 pho,fantastic pho -pron- vegetable pho vegetable b...,finely ground pork grill wooden skewer lettuce...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Get menu and text vectors per row

In [66]:
# Get menu and text vectors per text row 
menu_vectors = []
text_vectors = []

# Populating vectors per row
test_df['clean_text'].apply(lambda text: text_vectors.append(
    menu_doc2vec_model.infer_vector(doc_words=str(text).split(), epochs=5, alpha=0.025)))

test_df['menu'].apply(lambda text: menu_vectors.append(
    text_doc2vec_model.infer_vector(doc_words=str(text).split(), epochs=5, alpha=0.025)))

0       None
1       None
2       None
3       None
4       None
        ... 
1194    None
1195    None
1196    None
1197    None
1198    None
Name: menu, Length: 1199, dtype: object

#### Create menu and text dataframes consisting of its vectors

In [67]:
# Gather 100 vectors' name (0-99)
columns = np.arange(0,100)

# Create menu and text vectors dataframe
text_df = pd.DataFrame(data=text_vectors, columns=columns)
menu_df = pd.DataFrame(data=menu_vectors, columns=columns)

# Get name column (restaurant name) from test df
name_df = test_df[['name']]

# Concat name dataframe and menu/text vectors datafram
text_df = pd.concat([name_df, text_df], axis=1)
menu_df = pd.concat([name_df, menu_df], axis=1)

# Set Name column as index
menu_df = menu_df.set_index(['name'])
text_df = text_df.set_index(['name'])

In [68]:
# Text dataframe with vectors
text_df.head(3)

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
name,,,,,,,,,,,,,,,,,,,,,
#1 pho,-0.004585,-0.004472,-0.004607,0.000893,-0.000814,-0.003982,0.003827,0.002465,0.004907,0.003109,...,0.000658,-0.003352,-0.001305,0.004048,-0.004280,-0.003898,-0.001017,-0.003405,-0.000608,0.004354
24th street pizza & gyros,-0.039668,-0.013324,0.034346,0.055597,-0.025966,0.040305,0.011188,-0.005092,0.018282,-0.007482,...,0.036779,0.024018,0.015519,-0.012746,-0.044066,0.062545,0.051760,0.021928,-0.004965,-0.006115
3 tomatoes & a mozzarella,-0.003729,-0.005972,0.013928,0.014951,-0.013871,0.008267,-0.001188,-0.011447,0.002879,-0.004588,...,0.011047,0.012895,0.004184,-0.008633,-0.014957,0.020445,0.005603,0.010187,-0.004682,-0.004579


In [70]:
# Menu dataframe with vectors
menu_df.head(3)

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
name,,,,,,,,,,,,,,,,,,,,,
#1 pho,0.001005,-0.003237,-0.000196,-0.002253,-0.003255,0.001118,0.009194,0.002884,0.001176,0.003666,...,-0.004171,0.004614,0.004424,-0.002293,0.003283,-0.006279,0.004968,0.004755,-0.000483,0.001724
24th street pizza & gyros,-0.004757,-0.004305,-0.001718,-0.001710,0.002682,-0.003438,-0.002913,0.003903,-0.003586,-0.002949,...,0.001733,0.000696,-0.001166,0.000175,0.003181,-0.004477,0.000695,-0.004357,-0.004523,-0.002741
3 tomatoes & a mozzarella,-0.003171,0.003139,0.000530,0.004507,-0.003394,-0.003903,-0.001265,-0.000920,0.000605,-0.004101,...,0.003813,-0.003143,0.000721,-0.002170,-0.003733,0.001379,-0.003578,0.002343,0.000964,0.002612


#### Check if there are any NaNs for newly created menu and text dataframe

In [71]:
# Check if there are any NaNs
menu_df[menu_df.isnull().any(axis=1)]

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
name,,,,,,,,,,,,,,,,,,,,,


In [72]:
text_df[text_df.isnull().any(axis=1)]

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
name,,,,,,,,,,,,,,,,,,,,,


#### Save Menu and Text vectors dataframes

In [73]:
menu_df.to_csv('./dataset/menu_vectors_df.csv')
text_df.to_csv('./dataset/text_vectors_df.csv')

In [74]:
test_df.head()

,name,clean_text,menu,vegan,asian,ribs,fine,tapas,bistro,szechwan,...,fresh,puerto,gyros,cantonese,european,cream,cocktails,pub,indian,steak
0,#1 pho,fantastic pho -pron- vegetable pho vegetable b...,finely ground pork grill wooden skewer lettuce...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,24th street pizza & gyros,-pron- bad change ownership -pron- stop day ad...,tomato onion tzatziki sauce lettuce tomato oni...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3 tomatoes & a mozzarella,a cute unassume little bistro pizza go friday ...,small salad dress house balsamic vinaigrette r...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4b cafe,good consensus average home fairly easily emul...,stuff cream cheese freshly fry corn tortilla ...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5 r cha thai go,try r cha today turn permanently close very sa...,finely sliced veggie glass noodle wrap lightl...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [53]:
# Organize cuisine dataframes by utilizing dictionary.
categories_dict = {}
categories = test_df.columns[3:].tolist()
cols = test_df.columns[3:].tolist()
dfs = [] # Create master cuisine list

# Gather cuisine and key it by cuisine type onto dictionary
for category in categories:
    category_df = test_df[test_df[category] == 1]
    categories_dict[category] = category_df

In [96]:
def search(search_input):
    
    dfs = []
    search_input = tokenize(search_input)
    text_list = list(set(search_input.lower().split()))
    
    # Default dataframes
    text_result_df = text_df
    menu_result_df = menu_df
    
    # Only allow search when there is search term
    if len(text_list) > 0:
        
        # When there is keywords that matches
        for text in text_list:
            
            # Check if text is a key of categories dictionary
            if text in categories_dict:
                
                # Append to dfs list
                dfs.append(categories_dict[text])
                
            else:
                # Find most similar words using word2vec?
            
        # When search term(s) matches one of restarants' categories
        if len(dfs) > 0:
            
            result = None
            
            # Concat all dataframes
            if len(dfs) > 1:
                result_df = pd.concat(dfs)
            else:
                result_df = dfs[0]
            
            # Drop duplicates
            result_df = result_df.drop_duplicates(keep='first')
            result_df.reset_index(drop=True, inplace=True)
            
            # Get restaurant names
            restaurant_names = result_df['name'].tolist()
            
            text_result_df = text_df.loc[restaurant_names]
            menu_result_df = menu_df.loc[restaurant_names]
            
        # Test menu's doc2vec model
        menu_input_vectors = menu_doc2vec_model.infer_vector(search_input.split())

        menu_match_df = pd.DataFrame(menu_result_df.apply(lambda row: cos_similarity(menu_input_vectors, row.values), axis = 1), 
                                columns = ['distance']).reset_index(drop=True)

        # Test text's doc2vec model
        text_input_vectors = text_doc2vec_model.infer_vector(search_input.split())

        text_match_df = pd.DataFrame(text_result_df.apply(lambda row: cos_similarity(text_input_vectors, row.values), axis = 1), 
                                columns = ['distance']).reset_index(drop=True)

        # Add restaurant name to results dataframe
        menu_match_df['name'] = menu_result_df.index
        text_match_df['name'] = text_result_df.index

        # Sort cosine similarity distance by descending order
        menu_match_df = menu_match_df.sort_values(['distance'], ascending=False).reset_index(drop=True)
        text_match_df = text_match_df.sort_values(['distance'], ascending=False).reset_index(drop=True)
        
        return menu_match_df, text_match_df

In [174]:
# Testing functionalities
menu_result, text_result = search('pizza hut')

In [175]:
menu_result

,distance,name
0,0.252773,prime time pizza
1,0.241374,fazoli's
2,0.206722,arizona authentic pizza
3,0.182221,brothers pizza
4,0.151079,cucina rustica
...,...,...
137,-0.155666,cucina di vita
138,-0.159780,gus's new york pizza
139,-0.192789,girasole restaurant
140,-0.213261,bona pizza


In [176]:
text_result

,distance,name
0,0.122115,girasole restaurant
1,0.112136,captain tony's pizza
2,0.106863,circo
3,0.105440,pizza brutta
4,0.099113,pronto pizza
...,...,...
137,-0.084283,focaccia fiorentina
138,-0.108026,napoli pizza & pasta
139,-0.127060,prime time pizza
140,-0.127848,devil's pizza factory


In [104]:
menu_cuisine_df

,brazilian,british,cajun_creole,chinese,filipino,french,greek,indian,irish,italian,jamaican,japanese,korean,mexican,moroccan,russian,southern_us,spanish,thai,vietnamese
0,dried black beans,stilton,cajun,chinese,calamansi,gruyere cheese,feta cheese crumbles,garam masala,irish,lasagna,jerk,mirin,gochujang,taco,couscous,sauerkraut,grits,chorizo,thai,beansprouts
1,dried black,stilton cheese,cajun seasoning,oyster sauce,soy sauce bay,gruyere,cheese crumbles,garam,irish whiskey,lasagna noodles,jamaican,sake,gochujang base,taco seasoning,preserved lemon,buckwheat flour,whiskey,saffron threads,curry paste,thai
2,chocolate sprinkles,suet,andouille,shaoxing,oyster sauce,fresh tarragon,kalamata,curry leaves,irish cream liqueur,parmigiano,scotch bonnet,miso,kimchi,enchilada sauce,hanout,cottage cheese,bourbon,chorizo sausage,red curry,rice noodles
3,granola,currants,creole,hoisin,thai chile,swiss cheese,kalamata olives,dal,irish cream,parmigiano reggiano,bonnet chile,dashi,toasted sesame seeds,enchilada,ras,cottage,bourbon whiskey,spanish chorizo,red curry paste,vietnamese
4,ice lime,golden syrup,andouille sausage,hoisin sauce,thai,grated gruyère,pita,ghee,whiskey,reggiano cheese,scotch bonnet chile,nori,sauce sesame,jack cheese,ras el,buckwheat,collard greens,manchego cheese,lime leaves,thai basil
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,eggs parmesan,soda vanilla,onion powder dried,sesame oil rice,banana leaves,frisee,oregano black pepper,seeds cumin,fashioned,salt fresh basil,baking powder cornmeal,chili oil,sugar rice,ground cumin avocado,green olives olive,butter golden,chicken soup,salt manchego,sauce thai basil,sauce fresh ginger
96,passion fruit juice,sugar dark,powder dried oregano,sauce soy sauce,garlic coconut milk,chocolate large,yogurt garlic,salt ghee,wheat flour buttermilk,porcini,scallions coconut milk,soy sauce fresh,seeds green,flour tortillas salt,cinnamon lemon,potato starch,meal large,red peppers extra,paste lime,sauce fish
97,milk tapioca flour,potatoes all purpose,creole seasoning celery,sauce pork,sauce rice noodles,duck fat,pepper greek,jeera,carrots beef brisket,italian bread,scallions coconut,rice rice vinegar,reduced sodium soy,old el paso,lamb stock,potatoes all,cracker crumbs,tomatoes dry white,light soy,cilantro rice
98,ice granulated,sugar whipping cream,cajun seasoning all,rice vinegar corn,soy sauce vinegar,pernod,kalamata feta,gram flour,bacon pork sausages,cheese italian seasoning,thyme hot,green tea powder,soy sauce vegetable,el paso,ground coriander chopped,tomatoes potatoes,marshmallows,hot smoked paprika,light soy sauce,sesame oil rice


In [157]:
sentences = []
test_df['menu'].apply(lambda text: sentences.append(text.split()))

0       None
1       None
2       None
3       None
4       None
        ... 
1194    None
1195    None
1196    None
1197    None
1198    None
Name: menu, Length: 1199, dtype: object

In [159]:
menu_word2vec_model = Word2Vec(sentences, min_count=1, workers=4)

In [160]:
menu_word2vec_model.wv.vocab

{'finely': <gensim.models.keyedvectors.Vocab at 0x7f9124975130>,
 'ground': <gensim.models.keyedvectors.Vocab at 0x7f9105cae0a0>,
 'pork': <gensim.models.keyedvectors.Vocab at 0x7f9105cae610>,
 'grill': <gensim.models.keyedvectors.Vocab at 0x7f9105cae550>,
 'wooden': <gensim.models.keyedvectors.Vocab at 0x7f9105ca13a0>,
 'skewer': <gensim.models.keyedvectors.Vocab at 0x7f9105ca10a0>,
 'lettuce': <gensim.models.keyedvectors.Vocab at 0x7f9105ca11c0>,
 'rice': <gensim.models.keyedvectors.Vocab at 0x7f9105ca14c0>,
 'paper': <gensim.models.keyedvectors.Vocab at 0x7f9105ca12e0>,
 'pan': <gensim.models.keyedvectors.Vocab at 0x7f9105ca12b0>,
 'seared': <gensim.models.keyedvectors.Vocab at 0x7f9105ca1280>,
 'crepe': <gensim.models.keyedvectors.Vocab at 0x7f9105ca1790>,
 'bean': <gensim.models.keyedvectors.Vocab at 0x7f9105ca1670>,
 'sprout': <gensim.models.keyedvectors.Vocab at 0x7f9105ca1c10>,
 'lightly': <gensim.models.keyedvectors.Vocab at 0x7f9105ca1730>,
 'batter': <gensim.models.keyedvect

In [178]:
menu_word2vec_model.similar_by_vector('thai', topn=10)

[('siew', 0.6626031398773193),
 ('bangkok', 0.5868450403213501),
 ('proaw', 0.5733884572982788),
 ('padthai', 0.5337604880332947),
 ('augratin', 0.5289682149887085),
 ('paste', 0.5223985910415649),
 ('vermicelli', 0.5213367938995361),
 ('peanut', 0.5167734026908875),
 ('lemongrass', 0.5164087414741516),
 ('vietnamese', 0.5103117823600769)]

In [177]:
menu_cuisine_df

,brazilian,british,cajun_creole,chinese,filipino,french,greek,indian,irish,italian,jamaican,japanese,korean,mexican,moroccan,russian,southern_us,spanish,thai,vietnamese
0,dried black beans,stilton,cajun,chinese,calamansi,gruyere cheese,feta cheese crumbles,garam masala,irish,lasagna,jerk,mirin,gochujang,taco,couscous,sauerkraut,grits,chorizo,thai,beansprouts
1,dried black,stilton cheese,cajun seasoning,oyster sauce,soy sauce bay,gruyere,cheese crumbles,garam,irish whiskey,lasagna noodles,jamaican,sake,gochujang base,taco seasoning,preserved lemon,buckwheat flour,whiskey,saffron threads,curry paste,thai
2,chocolate sprinkles,suet,andouille,shaoxing,oyster sauce,fresh tarragon,kalamata,curry leaves,irish cream liqueur,parmigiano,scotch bonnet,miso,kimchi,enchilada sauce,hanout,cottage cheese,bourbon,chorizo sausage,red curry,rice noodles
3,granola,currants,creole,hoisin,thai chile,swiss cheese,kalamata olives,dal,irish cream,parmigiano reggiano,bonnet chile,dashi,toasted sesame seeds,enchilada,ras,cottage,bourbon whiskey,spanish chorizo,red curry paste,vietnamese
4,ice lime,golden syrup,andouille sausage,hoisin sauce,thai,grated gruyère,pita,ghee,whiskey,reggiano cheese,scotch bonnet chile,nori,sauce sesame,jack cheese,ras el,buckwheat,collard greens,manchego cheese,lime leaves,thai basil
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,eggs parmesan,soda vanilla,onion powder dried,sesame oil rice,banana leaves,frisee,oregano black pepper,seeds cumin,fashioned,salt fresh basil,baking powder cornmeal,chili oil,sugar rice,ground cumin avocado,green olives olive,butter golden,chicken soup,salt manchego,sauce thai basil,sauce fresh ginger
96,passion fruit juice,sugar dark,powder dried oregano,sauce soy sauce,garlic coconut milk,chocolate large,yogurt garlic,salt ghee,wheat flour buttermilk,porcini,scallions coconut milk,soy sauce fresh,seeds green,flour tortillas salt,cinnamon lemon,potato starch,meal large,red peppers extra,paste lime,sauce fish
97,milk tapioca flour,potatoes all purpose,creole seasoning celery,sauce pork,sauce rice noodles,duck fat,pepper greek,jeera,carrots beef brisket,italian bread,scallions coconut,rice rice vinegar,reduced sodium soy,old el paso,lamb stock,potatoes all,cracker crumbs,tomatoes dry white,light soy,cilantro rice
98,ice granulated,sugar whipping cream,cajun seasoning all,rice vinegar corn,soy sauce vinegar,pernod,kalamata feta,gram flour,bacon pork sausages,cheese italian seasoning,thyme hot,green tea powder,soy sauce vegetable,el paso,ground coriander chopped,tomatoes potatoes,marshmallows,hot smoked paprika,light soy sauce,sesame oil rice
